# Проект: Обучение с учителем: качество модели  
  
  
# Описание проекта  
  
Интернет-магазин «В один клик» продаёт разные товары: для детей, для дома, мелкую бытовую технику, косметику и даже продукты. Отчёт магазина за прошлый период показал, что активность покупателей начала снижаться. Привлекать новых клиентов уже не так эффективно: о магазине и так знает большая часть целевой аудитории. Возможный выход — удерживать активность постоянных клиентов. Сделать это можно с помощью персонализированных предложений.  
  
«В один клик» — современная компания, поэтому её руководство не хочет принимать решения просто так — только на основе анализа данных и бизнес-моделирования. У компании есть небольшой отдел цифровых технологий, и нам предстоит побыть в роли стажёра в этом отделе.   
  
Итак, нашему отделу поручили разработать решение, которое позволит персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность.  
  
# Цель исследования  
  
Нужно построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца.  
Используя данные модели и данные о прибыльности клиентов, нужно выделить сегменты покупателей и разработать для них персонализированные предложения.

# Краткий план исследования  
  
1. Загрузка и проверка данных
2. Предобработка данных
3. Исследовательский анализ данных
4. Объединение таблиц
5. Корреляционный анализ всех признаков
6. Использование пайплайнов
7. Анализ важности признаков
8. Сегментация покупателей
9. Общий вывод по исследованию

# Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
!pip install matplotlib==3.5 -q
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
import re

<class 'OSError'>: Not available

In [2]:
!pip install --upgrade scikit-learn -q
import sklearn
!pip install phik -q
import phik
!pip install shap -q
import shap

<class 'OSError'>: Not available

In [3]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

## Загрузим данные

In [4]:
df_market_file = pd.read_csv('/datasets/market_file.csv', sep=',')
df_market_money = pd.read_csv('/datasets/market_money.csv', sep=',')
df_market_time = pd.read_csv('/datasets/market_time.csv', sep=',')
df_money = pd.read_csv('/datasets/money.csv', sep=';')

<class 'FileNotFoundError'>: [Errno 44] No such file or directory: '/datasets/market_file.csv'

In [5]:
display(df_market_file.head(15))
df_market_file.info()

<class 'NameError'>: name 'df_market_file' is not defined

In [6]:
display(df_market_money.head(15))
df_market_money.info()

<class 'NameError'>: name 'df_market_money' is not defined

In [7]:
display(df_market_time.head(15))
df_market_time.info()

<class 'NameError'>: name 'df_market_time' is not defined

In [8]:
display(df_money.head(15))
df_money.info()

<class 'NameError'>: name 'df_money' is not defined

**Вывод:**  
  
**Мы загрузили данные и вывели первые строки и общую информацию каждого датафрейма**  
  
1. **df_market_file:**  
  - В таблице 1300 записей и 13 столбцов, пропуски в данных отсутствуют.  
  - В данных столбца тип занятости есть опечатка. Нужно будет ее исправить.  
  - В датафрейме присутствуют такие данные, как:  
    - **id** - номер покупателя в корпоративной базе данных.
    - **Покупательская активность** - рассчитанный класс покупательской активности (целевой признак): «снизилась» или «прежний уровень».
    - **Тип сервиса** - уровень сервиса, например «премиум» и «стандарт».
    - **Разрешить сообщать** - информация о том, можно ли присылать покупателю дополнительные предложения о товаре. Согласие на это даёт покупатель.
    - **Маркет_актив_6_мес** - среднемесячное значение маркетинговых коммуникаций компании, которое приходилось на покупателя за последние 6 месяцев. Это значение показывает, какое число рассылок, звонков, показов рекламы и прочего приходилось на клиента.
    - **Маркет_актив_тек_мес** - количество маркетинговых коммуникаций в текущем месяце.
    - **Длительность** - значение, которое показывает, сколько дней прошло с момента регистрации покупателя на сайте.
    - **Акционные_покупки** - среднемесячная доля покупок по акции от общего числа покупок за последние 6 месяцев.
    - **Популярная_категория** - самая популярная категория товаров у покупателя за последние 6 месяцев.
    - **Средний_просмотр_категорий_за_визит** - показывает, сколько в среднем категорий покупатель просмотрел за визит в течение последнего месяца.
    - **Неоплаченные_продукты_штук_квартал** - общее число неоплаченных товаров в корзине за последние 3 месяца.
    - **Ошибка_сервиса** - число сбоев, которые коснулись покупателя во время посещения сайта.
    - **Страниц_за_визит** - среднее количество страниц, которые просмотрел покупатель за один визит на сайт за последние 3 месяца.
  
2. **df_market_money:**  
  - В таблице 3900 записей и 3 столбца, пропусков так же нет.
  - В датафрейме присутствуют такие данные, как: 
    - **id** - номер покупателя в корпоративной базе данных.
    - **Период** - название периода, во время которого зафиксирована выручка. Например, 'текущий_месяц' или 'предыдущий_месяц'.
    - **Выручка** - сумма выручки за период.
  
3. **df_market_time:**  
  - В таблице 2600 строк и 3 столбца, пропусков в данных нет.  
  - В столбце с периодом тоже есть опечатки в значениях.  
  - В датафрейме присутствуют такие данные, как:  
    - **id** - номер покупателя в корпоративной базе данных.
    - **Период** -  название периода, во время которого зафиксировано общее время.
    - **минут** -  значение времени, проведённого на сайте, в минутах.
  
4. **df_money:**  
  - В таблице 1300 строк и 2 столбца, пропуски отсутствуют.
  - В столбце с прибылью неверной указан тип данных, нужно будет сменить его с object на float64.
  - В датафрейме присутствуют такие данные, как:  
    - **id** - номер покупателя в корпоративной базе данных.
    - **Прибыль** - значение прибыли.

## Предобработка данных

Наименования столбцов в датафреймах оставим русскими, только в таблице **df_money** поменяем разделитель значений в столбце **"Прибыль"** с **","** на **"."**. Так же изменим тип данных в столбце **"Прибыль"** с **object** на **float64**.

In [9]:
df_money['Прибыль'] = df_money['Прибыль'].str.replace(',', '.').astype('float64')

<class 'NameError'>: name 'df_money' is not defined

### Найдем явные и неявные дубликаты

In [10]:
display(df_market_file.duplicated().sum())
display(df_market_money.duplicated().sum())
display(df_market_time.duplicated().sum())
df_money.duplicated().sum()

<class 'NameError'>: name 'df_market_file' is not defined

Явные дубликаты отсутствуют.

#### Проверим уникальные значения типа object во всех столбцах.

In [11]:
print('Уникальные значения в столбце "Покупательская активность":', df_market_file['Покупательская активность'].unique())
print('Уникальные значения в столбце "Тип сервиса":', df_market_file['Тип сервиса'].unique())
print('Уникальные значения в столбце "Разрешить сообщать":', df_market_file['Разрешить сообщать'].unique())
print('Уникальные значения в столбце "Популярная_категория":', df_market_file['Популярная_категория'].unique())

<class 'NameError'>: name 'df_market_file' is not defined

В столбце с типом сервиса есть значение с опечаткой, исправим ее.

In [12]:
df_market_file['Тип сервиса'] = df_market_file['Тип сервиса'].str.replace('стандартт', 'стандарт')

<class 'NameError'>: name 'df_market_file' is not defined

In [13]:
print('Уникальные значения в столбце "Период":', df_market_money['Период'].unique())

<class 'NameError'>: name 'df_market_money' is not defined

В данном столбце нет опечаток и дубликатов.

In [14]:
print('Уникальные значения в столбце "Период":', df_market_time['Период'].unique())

<class 'NameError'>: name 'df_market_time' is not defined

Здесь присутствует опечатка в значении 'предыдущий месяц'

In [15]:
df_market_time['Период'] = df_market_time['Период'].str.replace('предыдцщий_месяц', 'предыдущий_месяц')

<class 'NameError'>: name 'df_market_time' is not defined

**Проверим, не появились ли явные дубликаты в ходе исправлений и все ли теперь правильно указано.**

In [16]:
display(df_market_file.duplicated().sum())
display(df_market_money.duplicated().sum())
display(df_market_time.duplicated().sum())
df_money.duplicated().sum()

<class 'NameError'>: name 'df_market_file' is not defined

In [17]:
display(df_market_file.info())
display(df_market_money.info())
display(df_market_time.info())
df_money.info()

<class 'NameError'>: name 'df_market_file' is not defined

**Вывод:**  
  
Мы устранили все опечатки и ошибки в данных. Явных дубликатов нет, пропуски тоже отсутствуют. Судя по данным везде количество клиентов одинаковое, просто где-то записей больше в 2-3 раза, поскольку по каждому клиенту информация указана например за 3 периода. Для объединения данных будем за основу брать столбец с id клиентов.

**Так же приведем названия столбцов к нижнему регистру.**

In [18]:
df_market_file.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', i).lower() for i in df_market_file.columns]
df_market_time.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', i).lower() for i in df_market_time.columns]
df_market_money.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', i).lower() for i in df_market_money.columns]
df_money.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', i).lower() for i in df_money.columns]
display(df_market_file.info())
display(df_market_time.info())
display(df_market_money.info())
df_money.info()

<class 'NameError'>: name 'df_market_file' is not defined

## Исследовательский анализ данных

### Выведем статистические данные по всем численным признакам, после построим гистограмы и "ящики с усами" и круговые диаграммы.

In [19]:
display(df_market_file[['маркет_актив_6_мес', 'маркет_актив_тек_мес', 'длительность', 'акционные_покупки', 
   'средний_просмотр_категорий_за_визит', 'неоплаченные_продукты_штук_квартал', 
   'ошибка_сервиса', 'страниц_за_визит']].describe())

<class 'NameError'>: name 'df_market_file' is not defined

In [20]:
display(df_market_money['выручка'].describe())

<class 'NameError'>: name 'df_market_money' is not defined

In [21]:
display(df_market_time['минут'].describe())

<class 'NameError'>: name 'df_market_time' is not defined

In [22]:
display(df_money['прибыль'].describe())

<class 'NameError'>: name 'df_money' is not defined

**Вывод:**  
  
Судя по показателям во всех данных датафреймов распределения довольно нормальные, везде медиана довольно близка с средним значением. Лишь в столбце с выручкой датафрейма df_market_money есть довольно высокое максимальное значение, дальше на графиках увидим подробнее, как оно влияет на качество данных.

In [23]:
def df_visualization(df):
    for column in df.columns:
        column_type = df[column].dtype
        
        if column_type in ['int64', 'float64'] and column !='id':
            fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize=(10, 5))
            axes = axes.ravel()
            sns.histplot(data = df[column], kde=True, bins = 30, ax=axes[0])
            axes[0].set_title(column)
            mean = df[column].mean()
            axes[0].axvline(mean, color = 'red', linestyle = '-', label = f'Mean: {mean:.2f}' )
            median = df[column].median()
            axes[0].axvline(median, color = 'yellow', linestyle = '--', label = f'Median: {median:.2f}')
            axes[0].legend()
            axes[0].legend(loc = 'upper right')
            
            sns.boxplot(x=df[column], ax=axes[1])
            axes[1].set_title(f'{column}')
            axes[1].set_xlabel(column)
            plt.show()

In [24]:
def df_visualization2(df):
    for column in df.columns:
        column_type = df[column].dtypes
        
        if column_type == 'object':
            value_counts = df[column].value_counts()
            value_counts.plot.pie(autopct='%1.2f%%')
            plt.title(f'{column}')
            plt.show()

In [25]:
df_visualization2(df_market_file)

<class 'NameError'>: name 'df_market_file' is not defined

In [26]:
display(df_visualization(df_market_money))
df_visualization2(df_market_money)

<class 'NameError'>: name 'df_market_money' is not defined

In [27]:
display(df_visualization(df_market_time))
df_visualization2(df_market_time)

<class 'NameError'>: name 'df_market_time' is not defined

In [28]:
df_visualization(df_money)

<class 'NameError'>: name 'df_money' is not defined

**Вывод:**  
  
Мы вывели гистограммы и "ящики с усами" по всем количественным признакам и круговые диаграммы по категориальным признакам. Во всех графиках довольно нормальное распределение, где-то присутствует распределение Пуассона. В датафрейме df_market_money в столбце "Выручка" присутствует явный выброс, думаю, стоит удалить во всех таблицах данного клиента, чтобы в дальнейшем эти данные не сказались плохо на результатах.

In [29]:
display(df_market_money['выручка'].describe())
df_market_money['выручка'].plot.hist(bins=30)
plt.title('Выручка')
plt.xlabel('Выручка')
plt.ylabel('Количество клиентов')
plt.show()

<class 'NameError'>: name 'df_market_money' is not defined

In [30]:
drop_id = df_market_money[df_market_money['выручка'] == 106862.2]['id']
df_market_file = df_market_file[~df_market_file['id'].isin(drop_id)]
df_market_money = df_market_money[~df_market_money['id'].isin(drop_id)]
df_market_time = df_market_time[~df_market_time['id'].isin(drop_id)]
df_money = df_money[~df_money['id'].isin(drop_id)]

<class 'NameError'>: name 'df_market_money' is not defined

In [31]:
df_market_file.info()
df_market_money.info()
df_market_time.info()
df_money.info()

<class 'NameError'>: name 'df_market_file' is not defined

In [32]:
df_market_money['выручка'].plot.hist(bins=30)
plt.title('Выручка')
plt.xlabel('Выручка')
plt.ylabel('Количество клиентов')
plt.show()

<class 'NameError'>: name 'df_market_money' is not defined

Клиента с аномальным значением в показателе "Выручка" убрали из всех датафреймов. 

### Теперь отберем клиентов с покупательской активностью не менее трёх месяцев

Данные о клиентах с покупательской активностью указаны только в одном датафрейме, это df_market_money. В нем есть такие признаки, как "Выручка" и "Период". Проверим, присутствуют ли клиенты с недостаточной покупательской активностью.

In [33]:
df_market_money[df_market_money['выручка'] == 0]

<class 'NameError'>: name 'df_market_money' is not defined

Всего 3 клиента, которые за последние 3 месяца не принесли прибыли. Поскольку в дальнейшем исследовании эти клиенты бесполезны, то стоит исключить их из данных.

In [34]:
drop_id2 = [215348, 215357, 215359]
df_market_file = df_market_file[~df_market_file['id'].isin(drop_id2)]
df_market_money = df_market_money[~df_market_money['id'].isin(drop_id2)]
df_market_time = df_market_time[~df_market_time['id'].isin(drop_id2)]
df_money = df_money[~df_money['id'].isin(drop_id2)]

<class 'NameError'>: name 'df_market_file' is not defined

In [35]:
df_market_file.info()
df_market_money.info()
df_market_time.info()
df_money.info()

<class 'NameError'>: name 'df_market_file' is not defined

### Общий вывод по исследовательскому анализу  
  
Мы провели анализ всех признаков во всех датафреймах, вывели гистограммы, "ящики с усами" и круговые диаграммы.  
  
Распределение большинства данных соответствует нормальному. Некоторые количественные признаки имеют распределение Пуассона. Так же из данных были удалены 4 клиента, 3 из которых за последние 3 месяца принесли сервису нулевую выручку, другой же клиент был исключен из данных по причине наличия аномального значения в признаке "Выручка".  
  
При анализе круговых диаграмм было замечено, что распределение некоторых категориальных признаков соответствует соотношению 1:3. Соотношение остальных признаков равномерное.

## Объединение таблиц

В основном датафрейме df_market_file всего 1296 строк, в df_market_money 3888 строк и в df_market_time 2592. Чтобы объединить эти датафреймы, их нужно привести к общему количеству строк.

In [36]:
df_market_money = df_market_money.pivot_table(index='id', columns='период', values='выручка')
df_market_money.columns = ['выручка_предыдущий_месяц', 'выручка_препредыдущий_месяц', 'выручка_текущий_месяц']
df_market_money.head(10)

<class 'NameError'>: name 'df_market_money' is not defined

In [37]:
df_market_time = df_market_time.pivot_table(index='id', columns='период', values='минут')
df_market_time.columns = ['минуты_предыдущий_месяц', 'минуты_текущий_месяц']
df_market_time.head(10)

<class 'NameError'>: name 'df_market_time' is not defined

In [38]:
merged_df = df_market_file.merge(df_market_money, on='id', how='inner')
display(merged_df.info())
merged_df.head()

<class 'NameError'>: name 'df_market_file' is not defined

In [39]:
df = merged_df.merge(df_market_time, on='id', how='inner')
display(df.info())
df.head()

<class 'NameError'>: name 'merged_df' is not defined

**Вывод:**  
  
Все таблицы успешно объединены, количество строк одинаковое и пропусков нет. В данном датафрейме теперь хранятся данные о всех актинвых пользователях за последние 3 месяца.

## Корреляционный анализ всех признаков

Нам необходимо провести корреляционный анализ признаков в количественной шкале в итоговой таблице для моделирования. Проверим зависимости между всеми признаками.  
  
Для начала заложим признаки с непрерывными значениями в переменную, чтобы при построении матрицы не было ошибок.

In [40]:
df_correlation = df.drop('id', axis=1)
interval_cols = df_correlation.select_dtypes(include=['int64', 'float64']).columns.tolist()

phik_matrix = df_correlation.phik_matrix(interval_cols=interval_cols)
plt.figure(figsize=(20, 15))
sns.heatmap(phik_matrix, annot=True)
plt.show()

<class 'NameError'>: name 'df' is not defined

**Вывод:**  
  
Мы вывели матрицу корреляции по всем количественным признакам и теперь можем увидеть все зависимости между целевым признаком и другими.  
  
Целовой признак имеет наиболее высокую корреляцию с следующими признаками:  
  
  - **Страниц_за_визит** - процент корреляции 0.75  
  - **минуты_предыдущий_месяц** - процент корреляции 0.69  
  - **минуты_текущий_месяц** - процент корреляции 0.58  
  - **Средний_просмотр_категорий_за_визит** - процент корреляции 0.54  
  - **Маркет_актив_6_мес** - процент корреляции 0.54  
  - **Акционные_покупки** - процент корреляции 0.51  
  - **Неоплаченные_продукты_штук_квартал** - процент корреляции 0.51  
  - **выручка_предыдущий_месяц** - процент корреляции 0.5  
  
Мультиколлинеарность между признаками не обнаружена.  

## Использование пайплайнов

### Пайплайн с подготовкой данных  
  
Во время подготовки данных будем использовать ColumnTransformer. Количественные и категориальные признаки обработаем в пайплайне раздельно. Для кодирования категориальных признаков будем использовать два кодировщика, для масштабирования количественных — два скейлера.

In [41]:
df_without_id = df.drop('id', axis=1)
df_without_id['покупательская активность'] = df_without_id['покупательская активность'].replace({'Снизилась': 1, 'Прежний уровень': 0})

target_attribute = ['покупательская активность']
features = df_without_id.columns.drop(target_attribute)

num_cols = df_without_id.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = df_without_id.select_dtypes(include=['object']).columns.tolist()

for i in cat_cols:
    unique_cat_cols = df_without_id[i].unique()
    print(f'Уникальные значения в столбце "{i}": {unique_cat_cols}')

print()
print('Все столбцы с количественными значениями:')
num_cols

<class 'NameError'>: name 'df' is not defined

**Создадим первый пайплайн с обработкой данных. Зафиксируем значения random_state и test_size и разделим датафрейм на тренировочные и тестовую выборки.**

In [42]:
RANDOM_STATE = 77
TEST_SIZE = 0.25

X_train, X_test, y_train, y_test = train_test_split(
    df_without_id.drop(target_attribute, axis=1),
    df_without_id[target_attribute],
    random_state=RANDOM_STATE,
    test_size=TEST_SIZE,
    stratify = df_without_id[target_attribute]
)

ohe_cols = ['разрешить сообщать', 'популярная_категория']
ord_cols = ['тип сервиса']
num_cols = ['маркет_актив_6_мес', 'маркет_актив_тек_мес', 'длительность', 'акционные_покупки', 
            'средний_просмотр_категорий_за_визит', 'неоплаченные_продукты_штук_квартал', 
            'ошибка_сервиса', 'страниц_за_визит', 'минуты_предыдущий_месяц', 'минуты_текущий_месяц']

ohe_pipe = Pipeline(
    [
        (
            'simpleImputer_ohe', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ohe', 
            OneHotEncoder(drop='first', handle_unknown='error')
        )
    ]
)

ord_pipe = Pipeline(
    [
        (
            'simpleImputer_before_ord', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ord',  
            OrdinalEncoder(
                categories=[
                    ['премиум', 'стандарт'], 
                ], 
                handle_unknown='use_encoded_value', unknown_value=np.nan
            )
        ),
        (
            'simpleImputer_after_ord', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        )
    ]
)

data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_cols),
        ('ord', ord_pipe, ord_cols),
        ('num', MinMaxScaler(), num_cols)
    ], 
    remainder='passthrough'
) 

data_preprocessor

<class 'NameError'>: name 'df_without_id' is not defined

In [43]:
pipe_final = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('models', DecisionTreeClassifier(random_state=RANDOM_STATE))
    ]
) 

<class 'NameError'>: name 'data_preprocessor' is not defined

Мы создали пайплайн с подготовкой данных и поместили его в итоговый вместе с моделью. Теперь создадим словарь с несколькими моделями и укажем гиперпараметры.

In [44]:
param_grid = [
    {
        # словарь для модели KNeighborsClassifier() 
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(1, 6),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    {
        # словарь для модели DecisionTreeClassifier()
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__max_depth': range(2, 10),
        'models__max_features': range(2, 10),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    {
        # словарь для модели LogisticRegression()
        'models': [LogisticRegression(random_state=RANDOM_STATE, solver='liblinear', penalty='l1')],
        'models__C': range(1, 8),
        'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']
    },
    {
        # словарь для модели SVC()
        'models': [SVC(random_state=RANDOM_STATE, probability=True)],
        'models__kernel': ('sigmoid', 'linear', 'rbf'),
        'preprocessor__num': [StandardScaler(), 'passthrough']
    }
]

Для анализа моделей будем использовать метрику ROC_AUC, поскольку она лучше справляется с дисбалансом классов.

In [45]:
randomized_search = RandomizedSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='roc_auc',
    random_state=RANDOM_STATE,
    n_jobs=-1
)

randomized_search.fit(X_train, y_train)

print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_)
print ('Метрика лучшей модели на тренировочной выборке:', randomized_search.best_score_)

<class 'NameError'>: name 'pipe_final' is not defined

In [46]:
probabilities = randomized_search.predict_proba(X_test)

probabilities_1 = probabilities[:, 1]
y_test_roc_auc = roc_auc_score(y_test, probabilities_1)

print(f'Метрика ROC-AUC на тестовой выборке для класса 1: {y_test_roc_auc}')

<class 'NameError'>: name 'randomized_search' is not defined

### Вывод по лучшей модели  
  
Самой лучшей моделью оказалась LogisticRegression с гиперпараметрами (C=1, penalty='l1', random_state=77, solver='liblinear').  
Ее качество на тренировочной выборке составляет 0.91, на тестовой 0.85 и при классовом предсказании она показала результат 0.89. Это довольно высокие показатели, значит модель редко делает ошибки, а значит ее можно использовать для предсказания покупательской активности.

## Анализ важности признаков

In [47]:
X_train_2 = pipe_final.named_steps['preprocessor'].fit_transform(X_train)
X_test_2 = pipe_final.named_steps['preprocessor'].transform(X_test)

explainer = shap.Explainer(randomized_search.best_estimator_.named_steps['models'], X_train_2)

feature_names = pipe_final.named_steps['preprocessor'].get_feature_names_out()

X_test_2 = pd.DataFrame(X_test_2, columns=feature_names)
shap_values = explainer(X_test_2)

shap.plots.bar(shap_values, max_display=50)

<class 'NameError'>: name 'pipe_final' is not defined

In [48]:
shap.plots.beeswarm(shap_values, max_display=250)

<class 'NameError'>: name 'shap' is not defined

**Вывод:**  
  
Мы вывели 2 графика, показывающих влияние признаков на целевой.  
  
Признаки, имеющие положительное значение SHAP на втором графике повышают вероятность снижения активности клиента, а признаки, имеющие отрицательное значение SHAP наоборот понижают вероятность снижения активности.
  
  
Судя по показателям почти никакого влияния не оказывают такие признаки, как Популярная_категория_Товары для детей, Макрет_актив_тек_мес и Ошибка_сервиса. Все эти 3 признака влияют на целевой меньше чем на 0.01. Соответственно их можно считайть бесполезными.    
  
**Более высокое влияние на целевой признак оказывают следующие признаки:**  
  
 - **Средний_просмотр_категорий_за_визит** - Данный признак показывает то, сколько различных категорий товаров пользователь просматривал в среднем за визит за последний месяц. Показатель его влияния на целевой признак равен 0.59 и на втором графике видно, что он имеет отрицательное значение.  
 - **минуты_предыдущий_месяц** - Этот признак показывает, сколько пользователь проводил времени на сайте за предыдущий месяц. Показатель влияния этого признака на целевой составляет 0.57. Значение SHAP так же отрицательное.  
 - **Страниц_за_визит** - Этот признак показывает, сколько пользователь просмотрел страниц за один визит. Показатель этого признака равен 0.55. У этого признака самое большое отрицательное значение SHAP.  
 - **минуты_текущий_месяц** - Этот признак показывает , сколько пользователь проводил времени на сайте за текущий месяц. Показатель влияния этого признака на целевой составляет 0.42. Значение SHAP отрицательное.  

## Сегментация покупателей

In [49]:
display(df.shape)
df.head()

<class 'NameError'>: name 'df' is not defined

In [50]:
display(df_money.shape)
df_money.head()

<class 'NameError'>: name 'df_money' is not defined

In [51]:
df = df.merge(df_money, on='id', how='inner')
display(df.shape)
df.head()

<class 'NameError'>: name 'df' is not defined

На этапе исследовательского анализа мы выяснили, что самыми популярными категориями являются Товары для детей и Домашний текстиль. Так что дальше будем работать с данным сегментом покупателей.  
  
Проведем графическое и аналитическое исследование групп покупателей и сделаем предложения по работе с сегментом для увеличения покупательской активности.

In [52]:
df_childrens_products = df[df['популярная_категория'] == 'Товары для детей']
display(df_childrens_products.shape)
df_childrens_products.head()

<class 'NameError'>: name 'df' is not defined

In [53]:
df_home_textiles = df[df['популярная_категория'] == 'Домашний текстиль']
display(df_home_textiles.shape)
df_home_textiles.head()

<class 'NameError'>: name 'df' is not defined

In [54]:
df_visualization(df_childrens_products)

<class 'NameError'>: name 'df_childrens_products' is not defined

In [55]:
df_visualization2(df_childrens_products)

<class 'NameError'>: name 'df_childrens_products' is not defined

In [56]:
df_visualization(df_home_textiles)

<class 'NameError'>: name 'df_home_textiles' is not defined

In [57]:
df_visualization2(df_home_textiles)

<class 'NameError'>: name 'df_home_textiles' is not defined

**Вывод по сегментации покупателей:**  
  
Проанализировав графики обоих сегментов мы видим, что покупательская активность чаще снижается, чем в общем датафрейме, а если точнее, то в общей выборке в 38% случаев покупательская активность снизилась, а в самых популярных категориях Товаря для детей и Домашний текстиль покупательская активность снизилась в 44% и 40% случаев соответственно.  
  
**Увеличить покупательскую активность в данных категориях покупателей можно следующими методами:**  
  - Для категории "Товары для детей" можно проводить сезонные распродажы, к примеру, к началу учебного года. Так же можно реализовать программу бонусов за покупки, делать дополнительные скидки в праздники и таким образом покупательская активность может возрасти.  
  - С категорией "Домашний текстиль" почти тоже самое, можно проводить распродажы в зависимости от сезона и так же предоставлять скидки, например, на наборы товаров, чтобы заинтересовать клиентов покупать большее количество товаров.

## Итоговый вывод по исследованию

**Описание задачи:**  
  
У нас была задача разработать решение, которое позволит персонализировать предложения постоянным клиентам, чтобы увеличить их покупательскую активность. Необходимо было построить модель, которая предскажет вероятность снижения покупательской активности клиента в следующие три месяца. Используя данные модели и данные о прибыльности клиентов, нужно было выделить сегменты покупателей и разработать для них персонализированные предложения.  
  
В исследовании мы использовали 4 таблицы с данными:  
 - В первой таблице была информация о поведении покупателя на сайте, о коммуникациях с покупателем и его продуктовом поведении. 
 - Вторая таблица включала в себя данные о выручке, которую получает магазин с покупателя, то есть сколько покупатель всего потратил за период взаимодействия с сайтом.  
 - В третьей таблицы были данные о времени (в минутах), которое покупатель провёл на сайте в течение периода.  
 - И последняя таблица содержала данные о среднемесячной прибыли покупателя за последние 3 месяца: какую прибыль получает магазин от продаж каждому покупателю.  
  
В данных остутствовали пропуски и так же не было дубликатов. Во всех таблицах была информация о 1300 пользователях. Было несколько опечаток, которые в итоге были исправленны и все таблицы объединены в один датафрейм.  
  
**Лучшая модель:**  
  
Лучшей моделью получилась Логистическая регрессия с такими гиперпараметрами как, random_state=77, C=1, penalty='l1', solver='liblinear'. Показатели метрик качества модели на тренировочной и тестовой выборке были равны 0.89.  
  
Наиболее влиятельными признаками на покупательскую активность оказались **"Средний_просмотр_категорий_за_визит"**, **"минуты_предыдущий_месяц"**, **"Страниц_за_визит"**, **"минуты_текущий_месяц"**.  
  
**Выводы и дополнительные предложения для выбранного сегмента покупателей:**  
  
Проанализировав графики обоих сегментов мы видим, что покупательская активность чаще снижается, чем в общем датафрейме, а если точнее, то в общей выборке в 38% случаев покупательская активность снизилась, а в самых популярных категориях Товаря для детей и Домашний текстиль покупательская активность снизилась в 44% и 40% случаев соответственно.  
  
Увеличить покупательскую активность в данных категориях покупателей можно следующими методами:  
  - Для категории "Товары для детей" можно проводить сезонные распродажы, к примеру, к началу учебного года. Так же можно реализовать программу бонусов за покупки, делать дополнительные скидки в праздники и таким образом покупательская активность может возрасти.  
  - С категорией "Домашний текстиль" почти тоже самое, можно проводить распродажы в зависимости от сезона и так же предоставлять скидки, например, на наборы товаров, чтобы заинтересовать клиентов покупать большее количество товаров.